# TP noté 

In [3]:
import requests
import pandas as pd

# Fonction pour récupérer les articles de PubMed
def get_pubmed_articles(keyword, max_results):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": keyword,
        "retmax": max_results,
        "retmode": "json",
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data.get("esearchresult", {}).get("idlist", [])

# Fonction pour obtenir les détails des articles, y compris les résumés
def get_article_details(article_ids):
    details_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    details_params = {
        "db": "pubmed",
        "id": ",".join(article_ids),
        "retmode": "xml",
        "rettype": "abstract",
    }
    response = requests.get(details_url, params=details_params)
    articles = []
    if response.status_code == 200:
        from xml.etree import ElementTree as ET
        root = ET.fromstring(response.content)
        for article in root.findall(".//PubmedArticle"):
            title = article.findtext(".//ArticleTitle", default="Titre non disponible")
            abstract = article.findtext(".//AbstractText", default="Résumé non disponible")
            journal = article.findtext(".//Journal/Title", default="Journal non disponible")
            pubdate = article.findtext(".//PubDate/Year", default="Date non disponible")
            article_id = article.findtext(".//ArticleId[@IdType='pubmed']", default="ID non disponible")
            articles.append({
                "Titre": title,
                "Résumé": abstract,
                "Journal": journal,
                "Date de publication": pubdate,
                "URL PubMed": f"https://pubmed.ncbi.nlm.nih.gov/{article_id}"
            })
    return articles

# Récupérer les articles et leurs détails
keyword = "stroke"
max_results = 10
article_ids = get_pubmed_articles(keyword, max_results)
articles = get_article_details(article_ids)

# Sauvegarder les résultats dans un fichier CSV
df = pd.DataFrame(articles)
df.to_csv("articles_pubmed_stroke.csv", index=False, encoding="utf-8")
print(f"\nLes articles ont été sauvegardés dans le fichier articles_pubmed_stroke.csv.")


Les articles ont été sauvegardés dans le fichier articles_pubmed_stroke.csv.
